In [1]:
import faulthandler 
from pyspark.sql import SparkSession 
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DateType, FloatType, TimestampType
from pyspark.sql.functions import col, array_contains, isnan, when, count
from pyspark.sql.functions import lit, concat_ws, concat, collect_list, udf
from pyspark.sql.functions import countDistinct
import plotly.express as px
from sklearn.preprocessing import RobustScaler
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import product
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import random
import itertools
from tqdm import tqdm
from scipy.stats import gmean

In [2]:

# spark = SparkSession.builder.master('local').appName('Python Spark SQL Practice').getOrCreate()
# Spark 세션 최적화
spark = SparkSession.builder \
    .appName("Ecommerce Reward Processing") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "4") \
    .config("spark.network.timeout", "600s") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

In [3]:
def search(dirname):
    filenames = os.listdir(dirname)
    file_list = []
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        print(full_filename)
        file_list.append(full_filename)
    return file_list
data_path_list = search("D:/workspace/data/")

D:/workspace/data/2019-Dec.csv
D:/workspace/data/2019-Nov.csv
D:/workspace/data/2019-Oct.csv
D:/workspace/data/2020-Apr.csv
D:/workspace/data/2020-Feb.csv
D:/workspace/data/2020-Jan.csv
D:/workspace/data/2020-Mar.csv


In [4]:
schema = StructType() \
      .add("event_time",TimestampType(),True) \
      .add("event_type",StringType(),True) \
      .add("product_id",StringType(),True) \
      .add("category_id",StringType(),True) \
      .add("category_code",StringType(),True) \
      .add("brand",StringType(),True) \
      .add("price",DoubleType(),True) \
      .add("user_id",StringType(),True) \
      .add("user_session",StringType(),True)

In [5]:
for i, x in enumerate(data_path_list):
    file_path = x
    df = spark.read.format("csv") \
      .option("header", True) \
      .option('delimiter', ',') \
      .schema(schema) \
      .load(file_path)
    if i == 0:
        merged_df = df
    else:
        merged_df = merged_df.union(df)

In [6]:
merged_df.show()

+-------------------+----------+----------+-------------------+--------------------+-------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|  brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-------+-------+---------+--------------------+
|2019-12-01 09:00:00|      view|   1005105|2232732093077520756|construction.tool...|  apple|1302.48|556695836|ca5eefc5-11f9-450...|
|2019-12-01 09:00:00|      view|  22700068|2232732091643068746|                NULL|  force| 102.96|577702456|de33debe-c7bf-44e...|
|2019-12-01 09:00:01|      view|   2402273|2232732100769874463|appliances.person...|  bosch| 313.52|539453785|5ee185a7-0689-4a3...|
|2019-12-01 09:00:02|  purchase|  26400248|2053013553056579841|computers.periphe...|   NULL| 132.31|535135317|61792a26-672f-4e6...|
|2019-12-01 09:00:02|      view|  20100164|2232732110089618156|    apparel.t

In [7]:
merged_df.createOrReplaceTempView("ecommerce")

In [8]:
purchases = spark.sql("""
    SELECT * 
    FROM ecommerce
    WHERE event_type = 'purchase'
""")
purchases.createOrReplaceTempView("purchases")

In [9]:
purchases = spark.sql("""
    SELECT *,
        CAST(event_time AS TIMESTAMP) AS event_timestamp,
        YEAR(event_time) AS year,
        MONTH(event_time) AS month,
        WEEKOFYEAR(event_time) AS week,
        DAY(event_time) AS day,
        HOUR(event_time) AS hour,
        MINUTE(event_time) AS minute,
        SECOND(event_time) AS second
    FROM purchases
""")
purchases.createOrReplaceTempView("purchases_transformed")

In [10]:
df_user_stats = spark.sql("""
    SELECT 
        user_id,
        SUM(price) AS total_spent,
        COUNT(product_id) AS total_purchases,
        AVG(price) AS avg_purchase_price,
        MIN(event_timestamp) AS first_purchase,
        MAX(event_timestamp) AS last_purchase,
        DATEDIFF(MAX(event_timestamp), MIN(event_timestamp)) AS days_between_purchases
    FROM purchases_transformed
    GROUP BY user_id
""")

In [ ]:
df_user_stats = spark.sql("""
    SELECT 
        user_id,
        SUM(price) AS total_spent,
        COUNT(product_id) AS total_purchases,
        AVG(price) AS avg_purchase_price,
        DATEDIFF(MAX(event_timestamp), MIN(event_timestamp)) AS days_between_purchases
    FROM purchases_transformed
    GROUP BY user_id
""")

In [11]:
# 매출 성장률 가정
growth_rates = [0.10, 0.20, 0.30]
reward_options = [1, 2, 5, 10]

In [12]:
simulation_results = []

# PySpark DataFrame → Pandas 변환 (for 문 사용을 위함)
user_stats_pd = df_user_stats.toPandas()

In [15]:
user_stats_pd.to_csv("전처리데이터for시뮬.csv", index=False)

In [13]:


# 시뮬레이션 실행 (GitHub 코드 방식과 동일)
for growth_rate in tqdm(growth_rates):
    for reward in reward_options:
        # 예상 매출 증가 적용
        user_stats_pd["new_revenue"] = user_stats_pd["total_spent"] * (1 + growth_rate)

        # 리워드 비용 적용 (1000명에게 지급 가정)
        user_stats_pd["reward_cost"] = reward * 1000

        # 순이익 계산
        user_stats_pd["net_profit"] = user_stats_pd["new_revenue"] - user_stats_pd["reward_cost"]

        # 수익성이 있는 경우만 저장
        profitable_rewards = user_stats_pd[user_stats_pd["net_profit"] > 0]
        simulation_results.append(profitable_rewards)

# 결과 병합
final_results = pd.concat(simulation_results, ignore_index=True)

100%|██████████| 3/3 [00:00<00:00,  4.82it/s]


In [17]:
final_results.to_csv("시뮬레이션 결과.csv")

In [46]:
purchases = spark.sql("""
    SELECT * 
    FROM ecommerce
    WHERE event_type = 'purchase'
""")
purchases.createOrReplaceTempView("purchases")

구매 이벤트(purchase)만 유지하여 데이터 크기 축소

In [47]:
purchases = spark.sql("""
    SELECT *,
        CAST(event_time AS TIMESTAMP) AS event_timestamp,
        YEAR(event_time) AS year,
        MONTH(event_time) AS month,
        WEEKOFYEAR(event_time) AS week,
        DAY(event_time) AS day,
        HOUR(event_time) AS hour,
        MINUTE(event_time) AS minute,
        SECOND(event_time) AS second
    FROM purchases
""")
purchases.createOrReplaceTempView("purchases_transformed")

event_time을 TIMESTAMP로 변환 후 연, 월, 주, 일, 시, 분, 초 컬럼 추가

In [48]:
user_stats = spark.sql("""
    SELECT 
        user_id,
        SUM(price) AS total_spent,
        COUNT(product_id) AS total_purchases,
        AVG(price) AS avg_purchase_price,
        MIN(event_timestamp) AS first_purchase,
        MAX(event_timestamp) AS last_purchase,
        DATEDIFF(MAX(event_timestamp), MIN(event_timestamp)) AS days_between_purchases
    FROM purchases_transformed
    GROUP BY user_id
""")
user_stats.createOrReplaceTempView("user_purchase_stats")

사용자별 총 구매액, 평균 구매 가격, 재구매 여부 분석

In [49]:
category_id_sales = spark.sql("""
    SELECT 
        category_id,
        SUM(price) AS total_sales,
        COUNT(product_id) AS total_purchases,
        AVG(price) AS avg_price
    FROM purchases_transformed
    GROUP BY category_id
""")
category_id_sales.createOrReplaceTempView("category_id_sales")

카테고리별 매출 분석 → 특정 제품군에 차등 리워드 적용 가능

In [50]:
df_session_stats = spark.sql("""
    SELECT 
        user_session,
        SUM(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) AS total_views,
        SUM(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) AS total_carts,
        SUM(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS total_purchases
    FROM ecommerce
    GROUP BY user_session
""")
df_session_stats.createOrReplaceTempView("session_stats")

# 전환율 계산
df_session_conversion = spark.sql("""
    SELECT *,
        (total_purchases / NULLIF(total_carts, 0)) AS cart_to_purchase_rate,
        (total_purchases / NULLIF(total_views, 0)) AS view_to_purchase_rate
    FROM session_stats
""")
df_session_conversion.createOrReplaceTempView("session_conversion")

 세션 내 전환율 분석 → 구매 전환율 낮은 고객에게 추가 리워드 제공 가능

In [51]:
growth_rates = [0.10, 0.20, 0.30]  # 매출 성장률 가정 (10%, 20%, 30%)
reward_options = [1, 2, 5, 10]  # 리워드 조합 (달러 단위)

# 성장률 & 리워드 옵션을 SQL 테이블로 생성
growth_rate_df = spark.createDataFrame([(g,) for g in growth_rates], ["growth_rate"])
reward_df = spark.createDataFrame([(r,) for r in reward_options], ["reward_amount"])

growth_rate_df.createOrReplaceTempView("growth_rates")
reward_df.createOrReplaceTempView("reward_options")

✅ 기존 연구 및 사례 기반으로 가정 설정
✅ 리워드 지급 후 매출 성장률(10%, 20%, 30%)을 가정하여 시뮬레이션

In [52]:
df_simulation = spark.sql("""
    SELECT 
        u.user_id,
        u.total_spent,
        r.reward_amount,
        g.growth_rate,
        u.total_spent * (1 + g.growth_rate) AS new_revenue,
        r.reward_amount * 1000 AS reward_cost,  -- 1000명 지급 가정
        (u.total_spent * (1 + g.growth_rate)) - (r.reward_amount * 1000) AS net_profit
    FROM user_purchase_stats u
    CROSS JOIN reward_options r
    CROSS JOIN growth_rates g
""")
df_simulation.createOrReplaceTempView("simulation_results")


In [22]:
from pyspark.sql.functions import broadcast

reward_df = spark.createDataFrame([(r,) for r in [1, 2, 5, 10]], ["reward_amount"])
growth_rate_df = spark.createDataFrame([(g,) for g in [0.10, 0.20, 0.30]], ["growth_rate"])

reward_df.createOrReplaceTempView("reward_options")
growth_rate_df.createOrReplaceTempView("growth_rates")

df_simulation = spark.sql("""
    SELECT /*+ BROADCAST(r, g) */
        u.user_id,
        u.total_spent,
        r.reward_amount,
        g.growth_rate,
        u.total_spent * (1 + g.growth_rate) AS new_revenue,
        r.reward_amount * 1000 AS reward_cost,  -- 1000명 지급 가정
        (u.total_spent * (1 + g.growth_rate)) - (r.reward_amount * 1000) AS net_profit
    FROM user_purchase_stats u
    JOIN reward_options r ON 1=1
    JOIN growth_rates g ON 1=1
""")


✅ 각 리워드 금액 & 성장률 조합에 대해 예상 매출 및 순이익 계산
✅ 리워드 비용을 고려한 수익성 유지 조합만 필터링

In [53]:
# 정렬 최적화 및 샘플링 적용
df_profitable = spark.sql("""
    SELECT * 
    FROM simulation_results
    WHERE net_profit > 0
    ORDER BY net_profit DESC NULLS LAST
""").sample(False, 0.1)  # 샘플링 적용

df_profitable = df_profitable.limit(10)  # LIMIT 10 적용

df_profitable.show(truncate=False)

Py4JJavaError: An error occurred while calling o556.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 428) (61.82.124.126 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

In [20]:
df_profitable = spark.sql("""
    SELECT * 
    FROM simulation_results
    WHERE net_profit > 0
    ORDER BY net_profit DESC
    LIMIT 10
""")
df_profitable.show(truncate=False)


Py4JJavaError: An error occurred while calling o193.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 427) (61.82.124.126 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

In [54]:
spark.stop()

In [ ]:
# trust_level별 확률 계산
trust_level_probs = df['trust_level'].value_counts(normalize=True)

# trust_level별 payment_value 분포 계산
trust_level_distributions = df.groupby('trust_level')['payment_value'].describe()

In [ ]:
stats = {}
for level in trust_level_distributions.index:
    stats[level] = {
        'mean': trust_level_distributions.loc[level, 'mean'],
        'std': trust_level_distributions.loc[level, 'std'],
        'min': trust_level_distributions.loc[level, 'min'],
        'max': trust_level_distributions.loc[level, 'max'],
    }
stats

In [ ]:
# 3. 통계값 기반 랜덤 값 생성 함수
def generate_random_value(trust, stats):
    mean = stats[trust]['mean']
    std = stats[trust]['std']
    min_val = stats[trust]['min']
    max_val = stats[trust]['max']
    
    # 정규분포에서 값 생성 및 min/max 제한
    value = np.random.normal(mean, std)
    value = np.clip(value, min_val, max_val)  # min/max 범위로 제한
    return value

성장율 = a %

리워드 조합 구성

랜덤 추출 해서 
new 수익 = 수익 * 성장율

리워드 조합에 따른 지출액

new 수익 - 지출액 > 0 되는 것 리워드 조합 찾기

In [25]:
spark.stop()